In [1]:
import torch
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
torch.cuda.is_available()

True

In [2]:
device = "cuda"

In [3]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
example = dataset1[0]
len(example)

2

In [4]:
img, label = example
img.shape


torch.Size([1, 28, 28])

In [33]:
img

tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0

In [5]:
img.shape, label

(torch.Size([1, 28, 28]), 5)

In [6]:
args = argparse.Namespace(batch_size=64, test_batch_size=64, lr=0.01, gamma=0.7, seed=1, epochs=1, no_cuda=False, no_mps=False, log_interval=10, save_model=False, dry_run=False)
train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
use_cuda = not args.no_cuda and torch.cuda.is_available()
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)


In [7]:
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
for batch in train_loader:
    break
imgs, preds = batch
imgs.shape, preds.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

In [10]:

for epoch in range(1, args.epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break
    test(model, device, test_loader)
    scheduler.step()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298794
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.270974
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.244571
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.149483
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.139151
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.012645
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.004800
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.897597
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.738846
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.608763
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.564437
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.336348
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.224592
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.349627
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.321006
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.156226
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.113709
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.059175
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.037540
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.773529
Train Epoch: 1 [12800/60000 (

In [34]:
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    break

In [43]:
optimizer.zero_grad()

In [36]:
params = list(model.parameters())
param = params[0]
len(params)

8

In [37]:
param.shape

torch.Size([32, 1, 3, 3])

In [38]:
param.grad

In [26]:
param.grad is None

True

In [39]:
output = model(data)
loss = F.nll_loss(output, target)
loss

tensor(2.2986, device='cuda:0', grad_fn=<NllLossBackward0>)

In [40]:
loss.backward()

In [41]:
param.grad is None

False

In [44]:
param.grad

In [42]:
param.grad.shape


torch.Size([32, 1, 3, 3])